## RE -> epsilon-NFA

(a+b)*abb
( recognize check stack[-1] +) ( push , a recognize check len(stack) check stack[-1]+ a push ,+ push(just push) , b recognize check len(stack) check stack[-1]+ stack pop+ b push stack push+ , ) traverse until ( reveals check stack[index'('-1]+ check index'('0 pop all(just concat) , * check len(stack) write , a check len(stack)write, b write, b write

(b+a(aa*b)*b)*


(b+aa+ac+aaa+aac)*

(1+01)*00(0+1)*

(0+1)*011  


In [2]:
from copy import deepcopy

In [ ]:
APT = 0 # a-zA-Z0-9
RE_OR = 1 # +
RE_CC = 2 # .
RE_ST = 3 # *
RE_OP = 4 # (
RE_CL = 5 # )

In [ ]:
def isapt(c):
    if ord(c) >= chr('a') and ord(c) <= chr('z') or ord(c) >= chr('A') and ord(c) <= chr('Z') or ord(c) >= chr('0') and ord(c) <= chr('9'):
        return True
    return False

In [2]:
class node:
    def __init__(self, cn, cv=None, left=None, right=None):
        self.cn = cn
        self.cv = cv
        self.left = None
        self.right = None

hello world


In [ ]:
RE = "(a+b)*abb"
stack = []
PostFix = ""

for c in RE:
    if isapt(c):
        PostFix += c
    elif c == '+':
        while stack and stack[-1] != '(':
            PostFix += stack.pop()
        stack.append('+')
    elif c == '.':
        while stack and stack[-1] != '(':
            PostFix += stack.pop()
        stack.append('.')
    elif c == '(':
        stack.append('(')
    elif c == ')':
        stack.append(')')
    elif c == '*':
        PostFix += c




for c in PostFix:
    if isapt(c):
        stack.append(node(APT, c))
    elif c == '+':
        tnd = node(RE_OR)
        tnd.left = stack.pop()
        tnd.right = stack.pop()
        stack.append(deepcopy(tnd))
    elif c == '.':
        tnd = node(RE_CC)
        tnd.left = stack.pop()
        tnd.right = stack.pop()
        stack.append(deepcopy(tnd))
    elif c == '*':
        tnd = node(RE_ST)
        tnd.left = stack.pop()
        stack.append(deepcopy(tnd))
    